# Aula6 - Doc2Query

[Unicamp - IA368DD: Deep Learning aplicado a sistemas de busca.](https://www.cpg.feec.unicamp.br/cpg/lista/caderno_horario_show.php?id=1779)

Autor: Marcus Vinícius Borela de Castro

[Repositório no github](https://github.com/marcusborela/deep_learning_em_buscas_unicamp)

Stage: finetuning t5-base doc2query

# Organizando o ambiente

In [1]:
import os

In [2]:
DIRETORIO_TRABALHO = '/home/borela/fontes/deep_learning_em_buscas_unicamp/local/doc2query'


In [3]:
assert os.path.exists(DIRETORIO_TRABALHO), f"Path para {DIRETORIO_TRABALHO} não existe!"

In [4]:
from psutil import virtual_memory

In [5]:
def mostra_memoria(lista_mem=['cpu']):
  """
  Esta função exibe informações de memória da CPU e/ou GPU, conforme parâmetros fornecidos.

  Parâmetros:
  -----------
  lista_mem : list, opcional
      Lista com strings 'cpu' e/ou 'gpu'. 
      'cpu' - exibe informações de memória da CPU.
      'gpu' - exibe informações de memória da GPU (se disponível).
      O valor padrão é ['cpu'].

  Saída:
  -------
  A função não retorna nada, apenas exibe as informações na tela.

  Exemplo de uso:
  ---------------
  Para exibir informações de memória da CPU:
      mostra_memoria(['cpu'])

  Para exibir informações de memória da CPU e GPU:
      mostra_memoria(['cpu', 'gpu'])
  
  Autor: Marcus Vinícius Borela de Castro

  """  
  if 'cpu' in lista_mem:
    vm = virtual_memory()
    ram={}
    ram['total']=round(vm.total / 1e9,2)
    ram['available']=round(virtual_memory().available / 1e9,2)
    # ram['percent']=round(virtual_memory().percent / 1e9,2)
    ram['used']=round(virtual_memory().used / 1e9,2)
    ram['free']=round(virtual_memory().free / 1e9,2)
    ram['active']=round(virtual_memory().active / 1e9,2)
    ram['inactive']=round(virtual_memory().inactive / 1e9,2)
    ram['buffers']=round(virtual_memory().buffers / 1e9,2)
    ram['cached']=round(virtual_memory().cached/1e9 ,2)
    print(f"Your runtime RAM in gb: \n total {ram['total']}\n available {ram['available']}\n used {ram['used']}\n free {ram['free']}\n cached {ram['cached']}\n buffers {ram['buffers']}")
    print('/nGPU')
    gpu_info = !nvidia-smi
  if 'gpu' in lista_mem:
    gpu_info = '\n'.join(gpu_info)
    if gpu_info.find('failed') >= 0:
      print('Not connected to a GPU')
    else:
      print(gpu_info)


In [6]:
mostra_memoria(['cpu','gpu'])

Your runtime RAM in gb: 
 total 67.35
 available 57.35
 used 8.94
 free 10.32
 cached 46.1
 buffers 2.0
/nGPU
Mon Apr 10 13:14:20 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.39.01    Driver Version: 510.39.01    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:02:00.0 Off |                  N/A |
| 65%   49C    P8    27W / 370W |     61MiB / 24576MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+--

## Fixando as seeds

In [7]:
import random
import torch
import numpy as np

In [8]:
def inicializa_seed(num_semente:int=123):
  """
  Inicializa as sementes para garantir a reprodutibilidade dos resultados do modelo.
  Essa é uma prática recomendada, já que a geração de números aleatórios pode influenciar os resultados do modelo.
  Além disso, a função também configura as sementes da GPU para garantir a reprodutibilidade quando se utiliza aceleração por GPU. 
  
  Args:
      num_semente (int): número da semente a ser utilizada para inicializar as sementes das bibliotecas.
  
  References:
      http://nlp.seas.harvard.edu/2018/04/03/attention.html
      https://github.com/CyberZHG/torch-multi-head-attention/blob/master/torch_multi_head_attention/multi_head_attention.py#L15
  """
  # Define as sementes das bibliotecas random, numpy e pytorch
  random.seed(num_semente)
  np.random.seed(num_semente)
  torch.manual_seed(num_semente)
  
  # Define as sementes da GPU
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = False

  #torch.cuda.manual_seed(num_semente)
  #Cuda algorithms
  #torch.backends.cudnn.deterministic = True


In [9]:
num_semente=123
inicializa_seed(num_semente)

## Preparando para debug e display

In [10]:
import pandas as pd

In [11]:
#!pip install transformers -q

In [12]:
import transformers

/home/borela/miniconda3/envs/treinapython39/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


https://zohaib.me/debugging-in-google-collab-notebook/

In [13]:
# !pip install -Uqq ipdb
import ipdb
# %pdb off # desativa debug em exceção
# %pdb on  # ativa debug em exceção
# ipdb.set_trace(context=8)  para execução nesse ponto

In [14]:
def config_display():
  """
  Esta função configura as opções de display do Pandas.
  """

  # Configurando formato saída Pandas
  # define o número máximo de colunas que serão exibidas
  pd.options.display.max_columns = None

  # define a largura máxima de uma linha
  pd.options.display.width = 1000

  # define o número máximo de linhas que serão exibidas
  pd.options.display.max_rows = 100

  # define o número máximo de caracteres por coluna
  pd.options.display.max_colwidth = 50

  # se deve exibir o número de linhas e colunas de um DataFrame.
  pd.options.display.show_dimensions = True

  # número de dígitos após a vírgula decimal a serem exibidos para floats.
  pd.options.display.precision = 7


In [15]:
def config_debug():
  """
  Esta função configura as opções de debug do PyTorch e dos pacotes
  transformers e datasets.
  """

  # Define opções de impressão de tensores para o modo científico
  torch.set_printoptions(sci_mode=True) 
  """
    Significa que valores muito grandes ou muito pequenos são mostrados em notação científica.
    Por exemplo, em vez de imprimir o número 0.0000012345 como 0.0000012345, 
    ele seria impresso como 1.2345e-06. Isso é útil em situações em que os valores dos tensores 
    envolvidos nas operações são muito grandes ou pequenos, e a notação científica permite 
    uma melhor compreensão dos números envolvidos.  
  """

  # Habilita detecção de anomalias no autograd do PyTorch
  torch.autograd.set_detect_anomaly(True)
  """
    Permite identificar operações que podem causar problemas de estabilidade numérica, 
    como gradientes explodindo ou desaparecendo. Quando essa opção é ativada, 
    o PyTorch verifica se há operações que geram valores NaN ou infinitos nos tensores 
    envolvidos no cálculo do gradiente. Se for detectado um valor anômalo, o PyTorch 
    interrompe a execução e gera uma exceção, permitindo que o erro seja corrigido 
    antes que se torne um problema maior.

    É importante notar que a detecção de anomalias pode ter um impacto significativo 
    no desempenho, especialmente em modelos grandes e complexos. Por esse motivo,
    ela deve ser usada com cautela e apenas para depuração.
  """

  # Configura variável de ambiente para habilitar a execução síncrona (bloqueante) das chamadas da API do CUDA.
  os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
  """
    o Python aguarda o término da execução de uma chamada da API do CUDA antes de executar a próxima chamada. 
    Isso é útil para depurar erros no código que envolve operações na GPU, pois permite que o erro seja capturado 
    no momento em que ocorre, e não depois de uma sequência de operações que pode tornar a origem do erro mais difícil de determinar.
    No entanto, é importante lembrar que esse modo de execução é significativamente mais lento do que a execução assíncrona, 
    que é o comportamento padrão do CUDA. Por isso, é recomendado utilizar esse comando apenas em situações de depuração 
    e removê-lo após a solução do problema.
  """

  # Define o nível de verbosity do pacote transformers para info
  # transformers.utils.logging.set_verbosity_info() 
  
  
  """
    Define o nível de detalhamento das mensagens de log geradas pela biblioteca Hugging Face Transformers 
    para o nível info. Isso significa que a biblioteca irá imprimir mensagens de log informativas sobre
    o andamento da execução, tais como tempo de execução, tamanho de batches, etc.

    Essas informações podem ser úteis para entender o que está acontecendo durante a execução da tarefa 
    e auxiliar no processo de debug. É importante notar que, em alguns casos, a quantidade de informações
    geradas pode ser muito grande, o que pode afetar o desempenho do sistema e dificultar a visualização
    das informações relevantes. Por isso, é importante ajustar o nível de detalhamento de acordo com a 
    necessidade de cada tarefa.
  
    Caso queira reduzir a quantidade de mensagens, comentar a linha acima e 
      descomentar as duas linhas abaixo, para definir o nível de verbosity como error ou warning
  
    transformers.utils.logging.set_verbosity_error()
    transformers.utils.logging.set_verbosity_warning()
  """


  # Define o modo verbose do xmode, que é utilizado no debug
  # %xmode Verbose 

  """
    Comando usado no Jupyter Notebook para controlar o modo de exibição das informações de exceções.
    O modo verbose é um modo detalhado que exibe informações adicionais ao imprimir as exceções.
    Ele inclui as informações de pilha de chamadas completa e valores de variáveis locais e globais 
    no momento da exceção. Isso pode ser útil para depurar e encontrar a causa de exceções em seu código.
    Ao usar %xmode Verbose, as informações de exceção serão impressas com mais detalhes e informações adicionais serão incluídas.

    Caso queira desabilitar o modo verbose e utilizar o modo plain, 
    comentar a linha acima e descomentar a linha abaixo:
    %xmode Plain
  """

  """
    Dica:
    1.  pdb (Python Debugger)
      Quando ocorre uma exceção em uma parte do código, o programa para a execução e exibe uma mensagem de erro 
      com informações sobre a exceção, como a linha do código em que ocorreu o erro e o tipo da exceção.

      Se você estiver depurando o código e quiser examinar o estado das variáveis ​​e executar outras operações 
      no momento em que a exceção ocorreu, pode usar o pdb (Python Debugger). Para isso, é preciso colocar o comando %debug 
      logo após ocorrer a exceção. Isso fará com que o programa pare na linha em que ocorreu a exceção e abra o pdb,
      permitindo que você explore o estado das variáveis, examine a pilha de chamadas e execute outras operações para depurar o código.


    2. ipdb
      O ipdb é um depurador interativo para o Python que oferece recursos mais avançados do que o pdb,
      incluindo a capacidade de navegar pelo código fonte enquanto depura.
      
      Você pode começar a depurar seu código inserindo o comando ipdb.set_trace() em qualquer lugar do 
      seu código onde deseja pausar a execução e começar a depurar. Quando a execução chegar nessa linha, 
      o depurador entrará em ação, permitindo que você examine o estado atual do seu programa e execute 
      comandos para investigar o comportamento.

      Durante a depuração, você pode usar comandos:
        next (para executar a próxima linha de código), 
        step (para entrar em uma função chamada na próxima linha de código) 
        continue (para continuar a execução normalmente até o próximo ponto de interrupção).

      Ao contrário do pdb, o ipdb é um depurador interativo que permite navegar pelo código fonte em que
      está trabalhando enquanto depura, permitindo que você inspecione variáveis, defina pontos de interrupção
      adicionais e até mesmo execute expressões Python no contexto do seu programa.
  """


In [16]:
config_display()

In [17]:
config_debug()

# Carga dos dados msmarco_triples.train.tiny.tsv

In [18]:
os.path.exists(f"{DIRETORIO_TRABALHO}/msmarco_triples.train.tiny.tsv")

True

In [19]:
if not os.path.exists(f"{DIRETORIO_TRABALHO}/msmarco_triples.train.tiny.tsv"):
    !wget https://storage.googleapis.com/unicamp-dl/ia368dd_2023s1/msmarco/msmarco_triples.train.tiny.tsv
    !mv msmarco_triples.train.tiny.tsv {DIRETORIO_TRABALHO}

In [20]:
df = pd.read_csv(f"{DIRETORIO_TRABALHO}/msmarco_triples.train.tiny.tsv", delimiter="\t", 
                 header=None, names=["query", "positive", "negative"])

In [21]:
df.shape

(11000, 3)

In [22]:
df.head()

,query,positive,negative
0,is a little caffeine ok during pregnancy,We donât know a lot about the effects of caf...,It is generally safe for pregnant women to eat...
1,what fruit is native to australia,Passiflora herbertiana. A rare passion fruit n...,"The kola nut is the fruit of the kola tree, a ..."
2,how large is the canadian military,The Canadian Armed Forces. 1 The first large-...,The Canadian Physician Health Institute (CPHI)...
3,types of fruit trees,Cherry. Cherry trees are found throughout the ...,"The kola nut is the fruit of the kola tree, a ..."
4,how many calories a day are lost breastfeeding,"Not only is breastfeeding better for the baby,...","However, you still need some niacin each day; ..."


In [23]:
df.dtypes


query       object
positive    object
negative    object
Length: 3, dtype: object

Verificando correção do arquivo!

In [24]:
print(df.isnull().sum())

query       0
positive    0
negative    0
Length: 3, dtype: int64


In [25]:
# !pip install -q ftfy
import ftfy

In [26]:
x = 'We donât know a lot about the effects'

In [27]:
ftfy.fix_text(x)

"We don't know a lot about the effects"

In [28]:
df.applymap(len).mean()

query        34.2256364
positive    353.7535455
negative    340.4646364
Length: 3, dtype: float64

In [29]:
# pois treinaremos doc2query apenas para geração de queries relevantes
del df['negative']

In [30]:
df['query'] = df['query'].apply(ftfy.fix_text)
df['positive'] = df['positive'].apply(ftfy.fix_text)


In [31]:
df.head()

,query,positive
0,is a little caffeine ok during pregnancy,We don't know a lot about the effects of caffe...
1,what fruit is native to australia,Passiflora herbertiana. A rare passion fruit n...
2,how large is the canadian military,The Canadian Armed Forces. 1 The first large-...
3,types of fruit trees,Cherry. Cherry trees are found throughout the ...
4,how many calories a day are lost breastfeeding,"Not only is breastfeeding better for the baby,..."


# Divisão em treino e validação

In [32]:
from sklearn.model_selection import train_test_split

In [33]:
X_train, X_valid, Y_train, Y_valid = train_test_split(df['positive'].values, 
                                                      df['query'].values,
                                                      test_size=1000, 
                                                      random_state=num_semente)

In [34]:
type(Y_valid), Y_valid.shape, Y_valid[0]

(numpy.ndarray, (1000,), 'how many fitbit steps equal a mile')

In [35]:
type(X_train), X_train.shape, X_train[0]

(numpy.ndarray,
 (10000,),
 'There are 40 weeks in a school year - and 12 weeks of holidays. Unless you live somewhere snowy and you have to take snow days. These can extend the length by another week or so. There are 40 weeks in a school year - and 12 weeks of holidays. Unless you live somewhere snowy and you have to take snow days.')

# Explorando o tokenizador do t5-base

In [36]:
from transformers import T5Tokenizer

In [37]:
tokenizer = T5Tokenizer.from_pretrained("t5-base")

/home/borela/miniconda3/envs/treinapython39/lib/python3.9/site-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [38]:
tokenizer.pad_token_id,tokenizer.cls_token_id,tokenizer.sep_token_id

(0, None, None)

In [39]:
x='it is just a test!'
y='it is just a continuation!'

In [40]:
input_ids = tokenizer.encode_plus(x,y)['input_ids']
print(input_ids)

[34, 19, 131, 3, 9, 794, 55, 1, 34, 19, 131, 3, 9, 25192, 55, 1]


In [41]:
print(tokenizer.batch_decode(input_ids))

['it', 'is', 'just', '', 'a', 'test', '!', '</s>', 'it', 'is', 'just', '', 'a', 'continuation', '!', '</s>']


In [42]:
input_ids = tokenizer([x,y])['input_ids']
print(input_ids)

[[34, 19, 131, 3, 9, 794, 55, 1], [34, 19, 131, 3, 9, 25192, 55, 1]]


In [43]:
tokenizer.eos_token, tokenizer.eos_token_id

('</s>', 1)

In [44]:
tokenizer.all_special_tokens[:4]

['</s>', '<unk>', '<pad>', '<extra_id_0>']

In [45]:
tokenizer.unk_token, tokenizer.unk_token_id

('<unk>', 2)

In [46]:
tokenizer.pad_token, tokenizer.pad_token_id

('<pad>', 0)

In [47]:
tokenizer.max_len_single_sentence, tokenizer.model_max_length

(511, 512)

# Criando dataset

In [48]:
from torch.utils.data import Dataset

In [49]:
from scipy import stats

In [50]:
class MyDataset(Dataset):
    """
      Classe para criar um dataset de texto e query.
    """  
    def __init__(self, texts: np.ndarray, queries:np.ndarray, tokenizer):
      """
      Inicializa um novo objeto MyDataset.

      Args:
          texts (np.ndarray): um array com as strings de texto. Cada linha deve ter 2 strings.
          tokenizer: um objeto tokenizer do Hugging Face Transformers.
          max_seq_length (int): o tamanho máximo da sequência a ser considerado.
      Raises:
          AssertionError: se os parâmetros não estiverem no formato esperado.
      """
      # Verifica se os parâmetros são do tipo esperado
      assert isinstance(texts, np.ndarray), f"Parâmetro texts deve ser do tipo np.ndarray e não {type(texts)}"
      assert isinstance(queries, np.ndarray), f"Parâmetro queries deve ser do tipo np.ndarray e não {type(queries)}"
      for row in texts:
          assert isinstance(row, str), f"Each element in texts.row must be a string e não {type(row)}"
          break

      self.max_seq_length = tokenizer.model_max_length

      # Salvar os dados dos tensores para adiantar o tempo de processamento
      self.tokenized_texts = tokenizer.batch_encode_plus(texts, return_length=True)
      self.tokenized_queries = tokenizer.batch_encode_plus(queries, return_attention_mask=False, return_length=True)
      
      print("tokenized_texts size stats:\n{}\n".format(stats.describe(self.tokenized_texts['length'])))
      print("tokenized_queries size stats:\n{}\n".format(stats.describe(self.tokenized_queries['length']))) 

    def __len__(self):
        """
          Retorna o tamanho do dataset (= tamanho do array texts)
        """
        return len(self.tokenized_texts["input_ids"])
    
    def __getitem__(self, index):
        """
          Retorna um dicionário com os dados do texto e sua classe correspondente, em um formato que pode 
          ser usado pelo dataloader do PyTorch para alimentar um modelo de aprendizado de máquina.
        """
        # print(f"getitem index={index} self.tokenized_texts['input_ids'][index] {self.tokenized_texts['input_ids'][index]}")
        saida = {"input_ids": self.tokenized_texts["input_ids"][index], 
                "attention_mask": self.tokenized_texts["attention_mask"][index], 
                "labels": self.tokenized_queries["input_ids"][index]} 
        # print(f"saida {saida}")
        return saida


#### Testando o MyDataset e o Dataloader

In [51]:
# Cria dados fictícios
texts = np.array(['This is the first text',
                  'This is text 2.1',
                  'This is text 3.1',
                  'This is text 4.1',
                  'This is text 5.1',
                  'This is text 6.1',
                  'This is text 7.1'])
queries = np.array(['This is the first query',
                  'This is query 2.1',
                  'This is query 3.1',
                  'This is query 4.1',
                  'This is query 5.1',
                  'This is query 6.1',
                  'This is query 7.1'])

In [52]:
texts.shape

(7,)

In [53]:
tokenizer.batch_encode_plus(texts)

{'input_ids': [[100, 19, 8, 166, 1499, 1], [100, 19, 1499, 3, 14489, 1], [100, 19, 1499, 3, 18495, 1], [100, 19, 1499, 3, 19708, 1], [100, 19, 1499, 3, 20519, 1], [100, 19, 1499, 3, 23769, 1], [100, 19, 1499, 3, 25059, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]}

In [54]:
# Cria um objeto da classe MyDataset
dummy_dataset = MyDataset(texts=texts, queries=queries, tokenizer=tokenizer)

tokenized_texts size stats:
DescribeResult(nobs=7, minmax=(6, 6), mean=6.0, variance=0.0, skewness=nan, kurtosis=nan)

tokenized_queries size stats:
DescribeResult(nobs=7, minmax=(6, 6), mean=6.0, variance=0.0, skewness=nan, kurtosis=nan)



/tmp/ipykernel_171944/1892734461.py:29: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  print("tokenized_texts size stats:\n{}\n".format(stats.describe(self.tokenized_texts['length'])))
/home/borela/miniconda3/envs/treinapython39/lib/python3.9/site-packages/scipy/stats/_stats_py.py:1522: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  sk = skew(a, axis, bias=bias)
/home/borela/miniconda3/envs/treinapython39/lib/python3.9/site-packages/scipy/stats/_stats_py.py:1523: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  kurt = kurtosis(a, axis, bias=bias)
/tmp/ipykernel_171944/1892734461.py:30: RuntimeWarning: Precision loss 

In [55]:
# Testa o método __len__()
assert len(dummy_dataset) == 7


In [56]:

# Testa o método __getitem__()
sample = dummy_dataset[0]


In [57]:
type(sample['input_ids'])

list

In [58]:

assert set(sample.keys()) == {'input_ids', 'attention_mask', 'labels'} # 
assert isinstance(sample['input_ids'], list)
assert isinstance(sample['attention_mask'], list)
assert isinstance(sample['labels'], list)


In [59]:
print(sample)

{'input_ids': [100, 19, 8, 166, 1499, 1], 'attention_mask': [1, 1, 1, 1, 1, 1], 'labels': [100, 19, 8, 166, 11417, 1]}


## Explorando dataset e dataloaser

In [60]:
from torch.utils.data import DataLoader

In [61]:
dummy_loader = DataLoader(dummy_dataset, batch_size=3, shuffle=False, num_workers=0 )

In [62]:
first_batch = next(iter(dummy_loader))

In [63]:
first_batch

{'input_ids': [tensor([100, 100, 100]),
  tensor([19, 19, 19]),
  tensor([   8, 1499, 1499]),
  tensor([166,   3,   3]),
  tensor([ 1499, 14489, 18495]),
  tensor([1, 1, 1])],
 'attention_mask': [tensor([1, 1, 1]),
  tensor([1, 1, 1]),
  tensor([1, 1, 1]),
  tensor([1, 1, 1]),
  tensor([1, 1, 1]),
  tensor([1, 1, 1])],
 'labels': [tensor([100, 100, 100]),
  tensor([19, 19, 19]),
  tensor([    8, 11417, 11417]),
  tensor([166,   3,   3]),
  tensor([11417, 14489, 18495]),
  tensor([1, 1, 1])]}

In [64]:
# train_dataset = MyDataset(X_train, Y_train, tokenizer)
val_dataset = MyDataset(X_valid, Y_valid, tokenizer)

tokenized_texts size stats:
DescribeResult(nobs=1000, minmax=(20, 299), mean=86.623, variance=1287.2781491491492, skewness=1.203754702156062, kurtosis=1.888567910351541)

tokenized_queries size stats:
DescribeResult(nobs=1000, minmax=(3, 25), mean=9.517, variance=10.392103103103105, skewness=0.8108317830729576, kurtosis=1.3237936614062669)



In [65]:
train_dataset = MyDataset(X_train, Y_train, tokenizer)

tokenized_texts size stats:
DescribeResult(nobs=10000, minmax=(14, 326), mean=86.9556, variance=1262.5688855285528, skewness=1.138437592548724, kurtosis=1.5776621817771685)

tokenized_queries size stats:
DescribeResult(nobs=10000, minmax=(3, 57), mean=9.5016, variance=12.030400480048003, skewness=1.7924405763231719, kurtosis=10.845883054521773)



In [66]:
len(train_dataset),len(val_dataset)

(10000, 1000)

# Treinamento

## Integração com Neptune

In [67]:
import getpass

In [68]:
os.environ['NEPTUNE_ALLOW_SELF_SIGNED_CERTIFICATE'] = 'TRUE'
os.environ['NEPTUNE_PROJECT'] = 'marcusborela/IA386DD'

In [69]:
os.environ['NEPTUNE_API_TOKEN'] = getpass.getpass('Informe NEPTUNE_API_TOKEN')


## Metrics definition

Código fonte em https://github.com/huggingface/transformers/blob/main/examples/pytorch/translation/run_translation.py

In [71]:
# !pip install evaluate
import evaluate

In [85]:
# Metric
metric = evaluate.load("sacrebleu")

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]
    return preds, labels

def compute_metrics_bleu(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)
    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    # print(f"Em compute_metrics: result={result}")

    result = {"bleu": result["score"]}
    print(f"Em compute_metrics: bleu={result['bleu']}")
    print(f"Predicao[0]    :{decoded_preds[0]}/nGround_truth[0]:{decoded_labels[0]}")

    return result
        

## Funções de apoio

In [73]:
from transformers import DataCollatorForSeq2Seq, T5ForConditionalGeneration

In [74]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, TrainerCallback

In [75]:
from transformers.optimization import Adafactor, AdafactorSchedule

In [76]:
from transformers import get_cosine_with_hard_restarts_schedule_with_warmup 

In [77]:
import time

In [78]:
import math

In [79]:
class CustomTrainerCallback(TrainerCallback):
    """
    Fonte inicial: Eduardo Seiti 
    """

    def __init__(self, best_validation_yet=99999, model=None) -> None:
        super().__init__()

        self.best_validation_metric = best_validation_yet
        self.model = model


    def on_evaluate(self, args, state, control, model=None, metrics=None, **kwargs):
        print(f'CustomTrainerCallback.on_evaluate - Momento: {time.strftime("[%Y-%b-%d %H:%M:%S]")} metrics={metrics}' )
        print(f"metrics['eval_loss']={metrics['eval_loss']} metrics['eval_bleu']={metrics['eval_bleu']} self.best_validation_metric={self.best_validation_metric}")

        if metrics['eval_bleu'] > self.best_validation_metric:
            self.best_validation_metric = metrics['eval_bleu']
        # caso queira salvar por aqui:
        #    nome_arquivo = f"{DIRETORIO_TRABALHO}/model-checkpoint-{state.global_step}-{metrics['eval_bleu']:.4f}"
        #    print(f"vou salvar {nome_arquivo}")
        #    self.model.save_pretrained(nome_arquivo)


In [80]:
def train(parm_model, # parm_optimizer, parm_lr_scheduler,
          num_batch_size:int=24, num_epochs:int=3, num_acum_steps:int=8):   # , num_steps_eval:int=100):
    """
    Função auxiliar de treinamento. 
    parm_model: o modelo Seq2Seq que será treinado.
    num_batch_size: o tamanho do lote (batch size) para treinamento e avaliação. Padrão é 24, o que significa que o valor será determinado pelo parâmetro per_device_train_batch_size em training_args.
    num_epochs: o número de épocas de treinamento. Padrão é 3.
    num_acum_steps: o número de passos de acumulação de gradiente. Padrão é 8.
    num_steps_eval: o número de passos para avaliação durante o treinamento. Padrão é 100.

    """
    global steps, diretorio, tokenizer, train_dataset, val_dataset


    num_training_steps = num_epochs * int(len(train_dataset) // (num_batch_size * num_acum_steps))
    # será avaliado a cada époica
    if num_epochs > 1:
        num_steps_eval = math.ceil(num_training_steps / num_epochs)  
    else:
        num_steps_eval = math.ceil(num_training_steps*0.1)
    print(f"num_training_steps = {num_training_steps} batch size = {num_batch_size} num_steps_eval={num_steps_eval}")

    trainer_callback = CustomTrainerCallback(best_validation_yet=-1, model=parm_model)


    # dicas em https://huggingface.co/transformers/v4.3.3/main_classes/trainer.html#seq2seqtrainingarguments 
    # Argumentos de treinamento do modelo Seq2Seq
    training_args = Seq2SeqTrainingArguments(
        output_dir=DIRETORIO_TRABALHO, # Onde os modelos são salvos
        logging_dir = DIRETORIO_TRABALHO+"/logs",
        # logging_strategy="steps",  # Especifique a estratégia de registro (por passos)
        logging_strategy="steps",  # Especifique a estratégia de registro (por passos)
        logging_steps=num_steps_eval, # Número de etapas para registrar os logs
        overwrite_output_dir=True,
        per_device_train_batch_size=num_batch_size, # Tamanho do batch por dispositivo durante o treinamento
        per_device_eval_batch_size=num_batch_size, # Tamanho do batch por dispositivo durante a avaliação
        gradient_accumulation_steps=num_acum_steps, # Número de etapas de acumulação de gradiente
        evaluation_strategy='steps', # Estratégia de avaliação durante o treinamento
        eval_steps=num_steps_eval, # Número de etapas para realizar a avaliação
        save_steps=num_steps_eval, # Em cada avaliação
        # lr_scheduler_type=None,  # Set to None to disable the default scheduler
        # fp16=True, # Usar precisão mista (half-precision) para acelerar o treinamento   
        bf16=True, # opção de configuração que permite o uso de aritmética de ponto flutuante de 16 bits (half-precision) 
        num_train_epochs=num_epochs, # Número de épocas de treinamento
        report_to="neptune",
        dataloader_pin_memory = True, # os dados carregados em memória pelo DataLoader são fixados (pinned) na memória do sistema 
                                      # Pode acelerar a transferência dos dados para a GPU,
                                      #  uma vez que a GPU pode ler os dados diretamente da memória fixada sem precisar fazer uma cópia adicional dos dados
        load_best_model_at_end=True, # Carregar o melhor modelo ao final do treinamento
        metric_for_best_model='bleu', # Métrica usada para selecionar o melhor modelo
        greater_is_better = True,
        # predict_with_generate é usado quando você deseja gerar sequências completas como saída do modelo,
        # enquanto do_predict é usado quando você deseja fazer predições em formato de passo a passo durante a inferência.
        predict_with_generate=True, # Permitir geração de predições com o modelo
        # do_predict=True, 
        warmup_steps = num_training_steps * 0.05, #  5% do total de passos 
        learning_rate=5e-3,
        #Implementa um aumento linear na taxa de aprendizado durante a fase de aquecimento (warmup) e, em seguida,
        # diminui linearmente a taxa de aprendizado após a fase de aquecimento. É uma escolha comum para tarefas de sequência para sequência.
        lr_scheduler_type= 'linear',
        weight_decay=1e-4,
        disable_tqdm=False, # not to disable the tqdm progress bars and table of metrics produced by NotebookTrainingTracker
        dataloader_drop_last = True, #r to drop the last incomplete batch (if the length of the dataset is not divisible by the batch size)
        save_total_limit=2 # Número máximo de checkpoints a serem salvos
    )
    
    # Objeto de collator de dados para ajustar os dados de treinamento
    data_collator = DataCollatorForSeq2Seq(
        tokenizer,
        model=parm_model,
        label_pad_token_id=-100, # Token de padding para os rótulos
        pad_to_multiple_of=8 if training_args.fp16 else None, # Valor de padding múltiplo de 8 para aproveitar otimizações com FP16
    )
    

    # Inicialização do treinador Seq2Seq
    trainer = Seq2SeqTrainer(
        model=parm_model,
        args=training_args,
        train_dataset=train_dataset, # Conjunto de dados de treinamento
        eval_dataset=val_dataset, # Conjunto de dados de avaliação
        data_collator=data_collator, # Collator de dados
        tokenizer=tokenizer, # Tokenizer
        callbacks=[trainer_callback], 
        # default to an instance of AdamW on your model and a scheduler given by get_linear_schedule_with_warmup() controlled by args
        # optimizers=(parm_optimizer, parm_lr_scheduler),
        compute_metrics=compute_metrics_bleu, # Função para calcular as métricas de avaliação
    )

    # Treinamento do modelo
    train_results = trainer.train()
    
    return trainer, train_results


In [82]:
num_batch_size=24
num_epochs=100
num_acum_steps=8

In [86]:
num_training_steps = num_epochs * int(len(train_dataset) // (num_batch_size * num_acum_steps))
if num_epochs > 1:
    num_steps_eval = math.ceil(num_training_steps / num_epochs)  
else:
    num_steps_eval = math.ceil(num_training_steps*0.1)
print(f"num_training_steps {num_training_steps} num_steps_eval {num_steps_eval} ")

num_training_steps 5200 num_steps_eval 52 


In [84]:
model = T5ForConditionalGeneration.from_pretrained("t5-base")

optimizer = Adafactor(model.parameters(), relative_step=False,lr=2e-4)
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-4, weight_decay=1e-3)
lr_scheduler = get_cosine_with_hard_restarts_schedule_with_warmup(optimizer, 
                                                              num_warmup_steps=10,
                                                              num_training_steps=num_training_steps, 
                                                              num_cycles=5)


#optimizer = Adafactor(model.parameters(), lr=2e-4, scale_parameter=True, relative_step=True, warmup_init=True)
optimizer = Adafactor(model.parameters(), relative_step=False,lr=2e-4)
lr_scheduler = AdafactorSchedule(optimizer)

In [87]:
trainer, train_results = train(parm_model=model, num_batch_size=num_batch_size, num_epochs=num_epochs, num_acum_steps=num_acum_steps)

num_training_steps = 5200 batch size = 24 num_steps_eval=52


/home/borela/miniconda3/envs/treinapython39/lib/python3.9/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


https://app.neptune.ai/marcusborela/IA386DD/e/IAD-72
Remember to stop your run once you’ve finished logging your metadata (https://docs.neptune.ai/api/run#stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.


/home/borela/miniconda3/envs/treinapython39/lib/python3.9/site-packages/transformers/integrations.py:1276: NeptuneDeprecationWarning: The object you're logging will be implicitly cast to a string. We'll end support of this behavior in `neptune-client==1.0.0`. To log the object as a string, use `str(object)` or `repr(object)` instead. For details, see https://docs.neptune.ai/setup/neptune-client_1-0_release_changes
  self._metadata_namespace[NeptuneCallback.model_parameters_key] = model.config.to_dict()
  1%|          | 52/5200 [05:38<9:15:55,  6.48s/it]

{'loss': 2.4423, 'learning_rate': 0.001, 'epoch': 1.0}


...

                                                      
100%|██████████| 5200/5200 [11:12:48<00:00, 11.85s/it]

Em compute_metrics: bleu=19.757756165764278
Predicao[0]    :how many steps does an average person walk per mile/nGround_truth[0]:['how many fitbit steps equal a mile']
{'eval_loss': 5.349058628082275, 'eval_bleu': 19.757756165764278, 'eval_runtime': 56.5867, 'eval_samples_per_second': 17.672, 'eval_steps_per_second': 0.742, 'epoch': 100.0}
CustomTrainerCallback.on_evaluate - Momento: [2023-Apr-11 00:29:06] metrics={'eval_loss': 5.349058628082275, 'eval_bleu': 19.757756165764278, 'eval_runtime': 56.5867, 'eval_samples_per_second': 17.672, 'eval_steps_per_second': 0.742, 'epoch': 100.0}
metrics['eval_loss']=5.349058628082275 metrics['eval_bleu']=19.757756165764278 self.best_validation_metric=20.092415224677104


100%|██████████| 5200/5200 [11:12:52<00:00, 11.85s/it]

{'train_runtime': 40376.6784, 'train_samples_per_second': 24.767, 'train_steps_per_second': 0.129, 'train_loss': 0.1323703731660946, 'epoch': 100.0}
Shutting down background jobs, please wait a moment...
Done!
Waiting for the remaining 14 operations to synchronize with Neptune. Do not kill this process.
All 14 operations synced, thanks for waiting!
Explore the metadata in the Neptune app:
https://app.neptune.ai/marcusborela/IA386DD/e/IAD-72


100%|██████████| 5200/5200 [11:12:53<00:00,  7.76s/it]


In [88]:
train_results

TrainOutput(global_step=5200, training_loss=0.1323703731660946, metrics={'train_runtime': 40376.6784, 'train_samples_per_second': 24.767, 'train_steps_per_second': 0.129, 'train_loss': 0.1323703731660946, 'epoch': 100.0})

In [90]:
trainer.evaluate()

100%|██████████| 41/41 [00:18<00:00,  2.15it/s]

Em compute_metrics: bleu=20.092415224677104
Predicao[0]    :how many steps does an average person walk per mile/nGround_truth[0]:['how many fitbit steps equal a mile']


/home/borela/miniconda3/envs/treinapython39/lib/python3.9/site-packages/transformers/integrations.py:1219: NeptuneDeprecationWarning: Parameter `run` is deprecated, use `with_id` instead. We'll end support of it in `neptune-client==1.0.0`.
  self._run = init_run(**self._init_run_kwargs, **additional_neptune_kwargs)


https://app.neptune.ai/marcusborela/IA386DD/e/IAD-72
Remember to stop your run once you’ve finished logging your metadata (https://docs.neptune.ai/api/run#stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.


100%|██████████| 41/41 [00:27<00:00,  1.48it/s]

CustomTrainerCallback.on_evaluate - Momento: [2023-Apr-11 07:07:42] metrics={'eval_loss': 5.336637020111084, 'eval_bleu': 20.092415224677104, 'eval_runtime': 26.8979, 'eval_samples_per_second': 37.178, 'eval_steps_per_second': 1.561, 'epoch': 100.0}
metrics['eval_loss']=5.336637020111084 metrics['eval_bleu']=20.092415224677104 self.best_validation_metric=20.092415224677104


{'eval_loss': 5.336637020111084,
 'eval_bleu': 20.092415224677104,
 'eval_runtime': 26.8979,
 'eval_samples_per_second': 37.178,
 'eval_steps_per_second': 1.561,
 'epoch': 100.0}

Error occurred during asynchronous operation processing: X-coordinates (step) must be strictly increasing for series attribute: finetuning/eval/bleu. Invalid point: 5200.0
Error occurred during asynchronous operation processing: X-coordinates (step) must be strictly increasing for series attribute: finetuning/eval/loss. Invalid point: 5200.0
Error occurred during asynchronous operation processing: X-coordinates (step) must be strictly increasing for series attribute: finetuning/eval/runtime. Invalid point: 5200.0
Error occurred during asynchronous operation processing: X-coordinates (step) must be strictly increasing for series attribute: finetuning/eval/samples_per_second. Invalid point: 5200.0
Error occurred during asynchronous operation processing: X-coordinates (step) must be strictly increasing for series attribute: finetuning/eval/steps_per_second. Invalid point: 5200.0


In [ ]:
CustomTrainerCallback.on_evaluate - Momento: [2023-Apr-11 07:07:42] metrics={'eval_loss': 5.336637020111084, 'eval_bleu': 20.092415224677104, 'eval_runtime': 26.8979, 'eval_samples_per_second': 37.178, 'eval_steps_per_second': 1.561, 'epoch': 100.0}
metrics['eval_loss']=5.336637020111084 metrics['eval_bleu']=20.092415224677104 self.best_validation_metric=20.092415224677104

{'eval_loss': 5.336637020111084,
 'eval_bleu': 20.092415224677104,
 'eval_runtime': 26.8979,
 'eval_samples_per_second': 37.178,
 'eval_steps_per_second': 1.561,
 'epoch': 100.0}

In [ ]:
# trainer.save_model()


metrics = trainer.evaluate()
print(metrics)